# Problem Set 4

In [1]:
from source_functions import *

In [2]:
df_3p, alphas_3p = load_mat_data('Simulation Data/100markets3products.mat', 3, 100)
df_5p, alphas_5p = load_mat_data('Simulation Data/100markets5products.mat', 5, 100)

In [14]:
#Arguments to be added to the function later
sigma_alpha_init = 1

### Argument: params_init
#params_init = jnp.concatenate([np.array([sigma_alpha_init]).reshape(-1,1), eta_init, delta_init], axis=0).flatten()

### argument: data
df = df_3p
alphas = alphas_3p

### argument: Mode 
mode = "demand_side"
#beta_true = (5, 1, 1)
#gamma_true = (2, 1, 1)
#alpha_true = 1
#sigma_true = 1
#J=3

In [19]:
#===============================================================================================================================================#
def estimate_BLP(df, sigma_alpha_init, mode, beta_true = (5, 1, 1), gamma_true = (2, 1, 1), alpha_true=1, sigma_alpha_true=1):

    ###
    J=3
    M = alphas.shape[0] #Number of markets
    N = alphas.shape[1] #Number of consumers

    if mode == "demand_side":
        N_instruments = 7
    elif mode == "p_exercise":
        N_instruments = 4
    elif mode == "supply_W":
        N_instruments = 8
    elif mode == "supply_joint":
        N_instruments = 8


    #Initial guesses for eta and delta
    eta_init =  jnp.ones((N_instruments, 1))
    delta_init = jnp.ones((J*M, 1))  

    #Initial guess for all parameters    
    params_init = jnp.concatenate([np.array([sigma_alpha_init]).reshape(-1,1), eta_init, delta_init], axis=0).flatten()


    MJN = (M, J, N_instruments, N) #Stack all shape-related parameters together

    #========== Constructing correct values of parameters, for use later ============================================================================#
    deltas_correct = (5 + df_3p['x2'] + df_3p['x3'] - df_3p['P_opt'] + df_3p['xi_all']).values.reshape(300, -1)
    # Prepend the new element
    first_elems = jnp.ones(1+N_instruments).reshape(-1, 1)
    params_correct = jnp.vstack([first_elems, deltas_correct]).reshape(-1)

    xi_true = (df_3p['xi_all']).values.reshape(300, -1)

    #========== Constructing commonly used variables ===============================================================================================#
    X = jnp.array(df_3p[['x1', 'x2', 'x3']].values) #Matrix of product characteristics
    prices = jnp.array(df_3p[['P_opt']].values)
    shares = jnp.array(df_3p[['shares']].values)
    W_costs = jnp.array(df_3p[['w']].values)


    # For speed, compute this outside of the function and pass it later
    alphas_repeat = jnp.repeat(np.array(alphas_3p.values), repeats=J, axis=0)

    # Random coefficients nu on the prices
    nus_on_prices = (alphas_repeat-1) * prices.reshape(-1, 1)

    # Get matrix of regressors
    Xbar = df_3p[['x1', 'x2', 'x3', 'P_opt']].values.reshape(J*M, 4)

    # Get the matrix of instruments (including x1, x2, and the BLP moments). 
    Z_everything = jnp.array(blp_instruments_all(X, W_costs, prices, MJN))

    if mode == "demand_side":
        Z = Z_everything[:, 0:7]
    elif mode == "p_exercise":
        Z = Z_everything[:, [0, 1, 2, 9]]
    elif mode == "supply_W":
        Z = Z_everything[:, 0:8]
    elif mode == "supply_joint":
        Z = Z_everything[:, 0:8]


    #Projection matrix onto the instruments
    Pz = Z @ jnp.linalg.inv(Z.T @ Z) @ Z.T
    #Annihiliator matrix to get xi from delta
    Az = jnp.eye(Pz.shape[0]) - Xbar @ jnp.linalg.inv(Xbar.T @ Pz @ Xbar) @ Xbar.T @ Pz
    #This thing gets the value of beta.
    M_iv_est = (np.linalg.inv(Xbar.T @ Pz @ Xbar) @ Xbar.T @ Pz)
    # GMM weighting matrix
    W = np.eye(N_instruments)



    #### Solve initial deltas for improved initial guess. 
    print("#============================================================================#")
    print("#== Getting feasible initial guess using modified MPEC")
    print("#============================================================================#")
    #res = solve_init_deltas(params_init, shares, nus_on_prices, MJN)





    #========== Define the constraints for the MPEC ===============================================================================================#
    constraints = [
        {
            'type': 'eq',  # Equality constraint g(x) = eta
            'fun': lambda x: np.asarray(constraint_g(x, Z, Az, MJN)),  # Convert to NumPy
            'jac': lambda x: np.asarray(constraint_g_jac(x, Z, Az, MJN))  # Convert Jacobian to NumPy
        },
        {
            'type': 'eq',  # Equality constraint s(x) = shares
            'fun': lambda x: np.asarray(constraint_s(x, shares, nus_on_prices, MJN)),
            'jac': lambda x: np.asarray(constraint_s_jac(x, shares, nus_on_prices, MJN))
        }
    ]
    if mode == "supply_joint":
        print("Supply-side version: Adding additional constraint to MPEC")
        #constraints.append(
            #New constraint needed for the supply side, joint MPEC
        #)


    #========== Running the MPEC optimization routine ==============================================================================================#

    print("#============================================================================#")
    print("#== Solving the MPEC optimization routine")
    print("#============================================================================#")


    # Silence warning about delta_grad==0.0
    warnings.filterwarnings("ignore", message="delta_grad == 0.0. Check if the approximated function is linear.")

    # Solve optimization
    tolerance = 1e-34
    W = np.eye(N_instruments)

    result = minimize(
        fun = lambda x: objective_mpec(x, W, MJN),
        x0 = params_init,
        constraints = constraints,
        method = 'trust-constr',
        jac = lambda x: objective_jac(x, W, MJN),
        hess = lambda x: objective_hess(x, W, MJN),
        options = {
                 'xtol': tolerance,
                 'gtol': tolerance,
                 'barrier_tol': tolerance,
                 'sparse_jacobian': True,
                 'disp': True,
                 'verbose': 3
                }, 
    )

    # Output results
    if result.success:
        print("Optimal solution found.")
    else:
        print("Optimization failed:", result.message)


    #========== Calculate the parameters of interest ==============================================================================================#
    theta_hat = result.x # Get the estimated value of theta
    delta_hat = theta_hat[N_instruments + 1:].reshape(300, 1)
    beta_and_alpha_hat = np.array(M_iv_est @ delta_hat)
    xi_hat = np.array(Az@delta_hat)

    beta_hat = beta_and_alpha_hat[:3]
    alpha_hat = -beta_and_alpha_hat[3].item()
    sigma_alpha_hat = theta_hat[0].item()


    print("#============================================================================#")
    print("#== Optimal parameters found. Next, calculating standard errors:")
    print("#== Calculating standard errors, elasticities, profits, and consumer surplus")
    print("#============================================================================#")

    #========== Calculate the standard errors =====================================================================================================#
    se_sigma, se_betas = standard_errors(theta_hat, Z, Az, M_iv_est, shares, nus_on_prices, MJN)
    se = np.append([se_sigma], se_betas)

    #========== Calculate the elasticities =====================================================================================================#
    #Predicted deltas, xis, and moment conditions
    beta_true_array = np.array(beta_true)
    #True value of the elasticities
    elasticities_true = calculate_price_elasticity(beta_true_array, alpha_true, sigma_alpha_true, xi_true, X, prices, shares, MJN)
    #Mean of the true value of elasticities
    mean_elasticities_true = elasticities_true.mean(axis=2)
    #######
    #Predicted value of the elasticities
    elasticities_hat = calculate_price_elasticity(beta_hat, alpha_hat, sigma_alpha_hat, xi_hat, X, prices, shares, MJN)
    #Mean of the true value of elasticities
    mean_elasticities_hat = elasticities_hat.mean(axis=2)

    #========== Calculate the marginal costs =====================================================================================================#
    mc_true = calculate_marginal_costs(elasticities_true, "oligopoly", prices, shares, MJN)
    mc_hat = calculate_marginal_costs(elasticities_hat, "oligopoly", prices, shares, MJN)

    #========== Calculate the consumer surplus ===================================================================================================#
    cs_true = calculate_consumer_surplus(beta_true_array, alpha_true, sigma_alpha_true, xi_true, X, prices, MJN)
    cs_hat = calculate_consumer_surplus(beta_hat, alpha_hat, sigma_alpha_hat, xi_hat, X, prices, MJN)



    #========== Create a dictionary to return all output ==========================================================================================#
    OUT = {
        'delta_hat': delta_hat,
        'beta_hat': beta_hat,
        'alpha_hat': alpha_hat,
        'sigma_alpha_hat': sigma_alpha_hat,
        'se': se,
        'elasticities': {'true': elasticities_true, 'hat': elasticities_hat}, 
        'mean_elasticities': {'true': mean_elasticities_true, 'hat': mean_elasticities_hat}, 
        'mc': {'true': mc_true, 'hat': mc_hat}, 
        'cs': {'true': cs_true, 'hat': cs_hat}, 
        'optim_results': result
    }

    return OUT

In [20]:
out=estimate_BLP(df, sigma_alpha_init, mode)

#============================================================================#
#== Getting feasible initial guess using modified MPEC
#============================================================================#
#============================================================================#
#== Solving the MPEC optimization routine
#============================================================================#
| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  | penalty  |CG stop|
|-------|-------|-------|-------------|----------|----------|----------|----------|-------|
|   1   |   1   |   0   | +7.0000e+00 | 1.00e+00 | 1.81e-04 | 1.00e+00 | 1.00e+00 |   0   |
|   2   |   2   |   1   | +4.5000e+00 | 5.60e+00 | 2.52e-04 | 8.02e-01 | 1.00e+00 |   1   |
|   3   |   3   |   2   | +1.8362e-01 | 3.14e+01 | 7.07e-05 | 4.13e-01 | 5.64e+01 |   1   |
|   4   |   4   |   3   | +6.6574e-02 | 1.12e+02 | 2.02e-04 | 1.06e-01 | 4.19e+02 |   1   |
|   5   |   5   |   4   | +4.3206e-

{'delta_hat': array([[-1.51823548],
        [ 5.58241611],
        [ 4.20948787],
        [ 5.41348196],
        [ 4.67015078],
        [ 6.27091601],
        [ 4.07252835],
        [ 3.65606122],
        [ 3.11711092],
        [ 0.09396839],
        [ 3.65429052],
        [ 2.3508348 ],
        [ 1.95897293],
        [ 3.75511544],
        [ 3.40180632],
        [ 1.75501574],
        [ 2.10559568],
        [ 1.46588402],
        [ 2.24099972],
        [ 1.76097478],
        [ 1.12260704],
        [-2.55757851],
        [ 2.6811035 ],
        [ 0.64950584],
        [ 0.33815914],
        [ 5.0300454 ],
        [ 5.49093588],
        [-0.09216978],
        [ 4.41065365],
        [ 0.13169566],
        [ 0.96265516],
        [ 1.99593287],
        [ 0.29533677],
        [ 1.94142685],
        [ 4.48338148],
        [ 3.6352318 ],
        [-0.72724448],
        [ 4.02847711],
        [ 2.75906319],
        [ 3.57148348],
        [ 3.04273542],
        [ 3.28123587],
        [ 0.80460819]